# MPCS 53111 - Homework 5

# Question 2

#### NOTICE:  I think CNNLayer should be just a layer on a CNN network, An object called Convolutional Neural Network should be responsible for taking in multiple images as parameters, while CNNLayer is just an object that comprises a CNN. So CNNLayer each time takes in just one image to be processed, not N images. So I'm not following the structures given by homework description, in this perspective. 

Code for the last homework has been used to define Neuron and FCLayer, being used for Fully Connected layer object. 

This is modified version of last homework which is to define Neuron and Fully-connected layer object. CNNLayer follows it. 

In [11]:
### this is Neuron class. 
import pandas as pd 
import numpy as np
import matplotlib as plt
  
class Neuron:
    # size of weights = num_nodes_last_layer*num_weight
    def __init__(self, weights_shape, bias=1,alpha=0.5):
 
        # self.label is the true label.
        self.label = None
        self.bias = bias
        # input is a list such that each item is a pd.DataFrame
        self.inputs = None
        self.predicted_label = None
        self.input_shape = None
        self.error = None
        self.true_label = None
        # learning_rate
        self.alpha = alpha
        # same size as self.inputs
        self.weights = None
        # same size as self.weights
        # should be passed back to previous layer        
        self.back_propagation_delta = None
        # size of weights is the same as size of a picture(1,M)
        self.weights_shape = weights_shape        
        # internal delta used to update weights
        self.delta = None 
        
        if self.weights is None:
            self.weights = (np.random.random([self.weights_shape[0],\
                           self.weights_shape[1],self.weights_shape[2]])-.5)*0.000000000001
                                                 
    def neuron_delta(self):
        # delta is a number type
        self.delta = self.error*self.derivative()
        
    # notitce: call this function before update weights!!!
    # returns a delta matrix that has then same size as self.inputs
    def back_propagation(self):
        self.neuron_delta()
        self.back_propagation_delta = self.error*self.delta.sum()
        return self.back_propagation_delta
 
    def calculate_error(self,y):
        self.true_label = y
        self.error = self.true_label - self.predicted_label
        
    def g(self, inputs):
        self.inputs = inputs
        self.input_shape = inputs.shape
        self.predicted_label = self.logistic_regression(self.calculate_in())
        self.output_shape = (self.input_shape[0],1)
  
        # should be of shape (inputs[0],1)
        return self.predicted_label

    # calculate 
    def calculate_in(self):                
        # inputs[1] and weights[1] are DataFrame
        return np.dot(self.inputs, self.weights).sum() + self.bias

    # this is g'(in)
    def derivative(self,epsilon = 0.001):   
        return (self.logistic_regression(self.calculate_in()+epsilon) - \
                self.logistic_regression(self.calculate_in()-epsilon))/(2*epsilon)

    # logistic function
    def logistic_regression(self, z):
        return 1 / (1 + np.exp(-z))        

    # notice: don't call it until calling back_propagation !
    def update_weights(self):
        if self.delta is None:
            print("Call self.neuron_delta() first to compute delta of this Neuron")
            return
        self.weights = self.weights + self.alpha*self.delta*self.inputs

In [12]:
# each Layer object contains neuron_num numbers of neuron objects
# this FCLayer is different from ANN.
class FCLayer:
    # neuron_num is the number of neurons within this layer
    # weight_num is how many weights for each neuron
    def __init__(self, alpha,neuron_num,weights_shape, true_label, bias=1):
        self.alpha = alpha
        self.true_label = true_label
        # default bias = 1
        self.bias = bias 
        # neurons 
        self.neurons = []
        # how many neurons 
        self.neuron_num= neuron_num
        # self.output is the output that feeds to the next layer
        self.output = None
        # contains deltas from all neurons in layer
        # shape should be # of neurons+self.input.shape
        self.layer_deltas  = []
        # this is the final delta matrix passed back to previous CNNLayer
        self.layer_delta  = []
        self.weights_shape = weights_shape
        
        for i in range(neuron_num):
            self.neurons.append(Neuron(self.weights_shape,self.bias))

    # calculate the output for each neuron
    # store it in self.output
    def feed_forward(self, inputs):
        self.inputs = inputs
        
        self.output = []
        for neuron in self.neurons:
            self.output.append(neuron.g(inputs))
        self.output = np.array(self.output)
       
    # return output to next layer
    # size should be the same self.inputs
    def back_propagation(self):
        # must call calculate_error in each neuron first 
        # then neuron_delta can be called.
        # assign true_label to each neuron in layer
        for i in range(len(self.true_label)):
            self.neurons[i].calculate_error(self.true_label[i])
        self.layer_deltas = []
        
        for neuron in self.neurons:
            self.layer_deltas.append(neuron.back_propagation())
  
        self.layer_deltas = np.array(self.layer_deltas)      

        # passed to previous layer
        self.layer_delta = np.zeros(self.inputs.shape)        
        for i in range(len(self.layer_deltas)):
            self.layer_delta = self.layer_delta + self.layer_deltas[i]*self.neurons[i].weights
            
        return self.layer_delta
    # call after calling back_propagation()
    def update_weights(self):
        for i in range(len(self.neurons)):
            self.neurons[i].update_weights()
            
    def set_true_label(self,true_label):
        self.true_label=true_label    

In [13]:
# this is CNNLayer Object
# the following CNNLayer class is new 
class CNNLayer:
    
    # notice: order of filter shape should be (depth,height,width)
    #          order of filter shape should be (height,width,depth)  
    def __init__(self,n=32,filter_shape=(4,5,5),activation="relu",stride=2,alpha=100):
        self.bias = 1
        self.input = input
        self.filter_shape = tuple([n]+list(filter_shape))
        self.alpha = alpha
        self.stride = stride
        # contains filter arrays
        self.filters = None
        # each image's shape
        self.output_shape = None
        # layer_delta is the delta matrix passed back to previous layer
        self.layer_delta = None
        # this is the delta matrix from next layer
        self.next_layer_delta =None
        # self.output is the output by forward computation going to next layer
        self.output = None
        # number of filters
        self.filter_layer = n
        # input with 0 being padded
        self.padded_input = None
        # bias is a array of size n, same as the number of filters
        # for now just randomly choose from 1 and 0
        self.bias=np.random.choice([0,1],n)
        
        if activation == "no_act":
            self.activation = self._noact
        elif activation == "relu":
            self.activation = self._ReLU   
            
        # notice that the filters are defines as [depth,width,height] rather
        # than [width,height,depth]
        self.filters = np.random.choice([-1,0,1],n*filter_shape[0]*filter_shape[1]*\
                filter_shape[2]).reshape([n,filter_shape[0],filter_shape[1],\
                                         filter_shape[2]])
                         
    # Notice: the padding functionality assumes that the input images
    # are square size, which means width and height are of same size. 
    # Be lazy here.   
    def _padding(self,pad =0):
        
        output_size=int(np.ceil((self.input.shape[2]-self.filters[0].shape[2])/self.stride+1))       
        # output size for whole set of image
        self.output_shape=(self.filter_layer,output_size,output_size)
        # output size for the whole set of images
        self.output = np.zeros([self.filter_layer,output_size,output_size])
        
        # calculating number of rows/cols to pad. 
        # padding_length = 1 means add 1 row and column to surface of self.input
        self.padding_length = int(np.ceil((self.output_shape[1]-1)*self.stride - \
                          self.input.shape[2] + self.filter_shape[2]))

        # create a new image with 0 being padded
        if self.padding_length > 0:
            new_images = []
    
            for each_depth in self.input:
                new_image = np.insert(each_depth,len(each_depth),\
                                           np.zeros(each_depth.shape[0]),axis=1)
                new_image  = np.insert(new_image,len(new_image),\
                                       np.zeros(new_image.shape[1]),axis=0)
                
                for i in range(self.padding_length-1):
                    new_image = np.insert(new_image,len(new_image),\
                                           np.zeros(new_image.shape[0]),axis=1)
                    new_image  = np.insert(new_image,len(new_image),\
                                       np.zeros(new_image.shape[1]),axis=0)
                
                new_images.append(new_image)               
            self.padded_input = np.array(new_images)         
        else: 
            self.padded_input = self.input
        # END of padding
        # create output array
        self.output = np.zeros([self.filter_layer,(self.padded_input.shape[1]-\
                       self.filters[0].shape[1])/self.stride+1,\
                       (self.padded_input.shape[1]-self.filters[0].shape[1])/self.stride+1])
                        
    def forward_step(self,X):        
        if X.shape[0] != self.filter_shape[1]:
            print("filter depth does not match image depth!")
        
        # input image        
        self.input = X     
        self._padding(self.input)
        
        # forward computation
        for k in range(self.output_shape[0]):
            current_filter = self.filters[k]

            for i in range(self.output_shape[1]):
                for j in range(self.output_shape[2]):
                    self.output[k,i,j] =  self.activation(1+(current_filter*\
                            self.padded_input[:,self.stride*i:self.stride*i+\
                          current_filter.shape[1],self.stride*j:self.stride*j+\
                            current_filter.shape[2]]).sum())                      
        return self.output
    # by default, there should be random filters assigned for debugging purpose 
    # so even if you don't set filters, self.filters will still be there
    
    def set_filters(self,filters):   
        if filters.shape != self.filter_shape:
            print("filters must be numpy array with shape as",self.filter_shape)        
        else:
            self.filters = filters
            
    # set bias
    def set_bias(self,bias):   
        if bias.shape != self.bias.shape:
            print("filters must be numpy array with shape as",self.bias.shape)        
        else:
            self.bias = bias
    
    # print filters
    def print(self):
        print("Filter shape:",self.filter_shape)
        
        for i in range(len(self.filters)):
            print("Filter layer",i+1,":")
            print(self.filters[i],"\n")

    def _noact(self,x):
        return x                
    def _ReLU(self,x):
        return max(0,x)
        
    # this is g'(in)
    def derivative(self,f,data,epsilon = 0.001):
        return (f(data+epsilon) - f(data-epsilon))/(2*epsilon)                 
    
    def backward_step_from_fc(self,next_layer_delta):
        # assert
        if next_layer_delta.shape != self.output.shape:
            print("Delta Matrix received from next layer does not match current output!") 
            return 
        
        self.next_layer_delta = next_layer_delta
        # compute g prime
        f_add_h = np.zeros(self.output.shape)
        f_minus_h = np.zeros(self.output.shape)
        h = 0.001
        
        # f_add_h f_minus_h 
        for k in range(self.output_shape[0]):
            current_filter = self.filters[k]

            for i in range(self.output_shape[1]):
                for j in range(self.output_shape[2]):
                    f_add_h[k,i,j] =  self.activation(1+(current_filter*\
                            self.padded_input[:,self.stride*i:self.stride*i+\
                          current_filter.shape[1],self.stride*j:self.stride*j+\
                            current_filter.shape[2]]).sum()+h) 
                    
                    f_minus_h[k,i,j] =  self.activation(1+(current_filter*\
                            self.padded_input[:,self.stride*i:self.stride*i+\
                          current_filter.shape[1],self.stride*j:self.stride*j+\
                            current_filter.shape[2]]).sum()-h) 
        # g_prime
        self.g_prime = (f_add_h - f_minus_h)/2*h
        self.delta_j = self.next_layer_delta*self.g_prime
      
        # compute delta matrix passed to previous layer
        self.layer_delta = np.zeros(self.delta_j.shape[0])
        
        for k in range(len(self.filters)):
            self.layer_delta[k] = (self.delta_j*self.filters[k]).sum()
        return self.layer_delta
 
        
    def backward_step_from_cnn(self,next_layer_delta):      
        
        self.next_layer_delta = next_layer_delta
        self.new_layer_delta = np.zeros(self.filter_shape)
        
        for i in range(len(self.next_layer_delta)):
           self.new_layer_delta[i] = self.next_layer_delta[i]

        self.next_layer_delta = self.new_layer_delta
        
        self.new_output =np.zeros(self.output.shape[0])
        for i in self.output:
            self.new_output = i.sum()

        self.output = self.new_output
            
    def update_weights_from_fc(self):
        
        self.new_filters = np.zeros(self.filters.shape)        
        for i in range(len(self.filters)):
            self.new_filters = self.filters + self.alpha*self.next_layer_delta[i]*self.output
        self.filters = self.new_filters

    def update_weights_from_cnn(self):
        self.filters = self.filters + self.alpha*self.next_layer_delta*self.output
        

In [14]:
##### quick test on the shapes of images

In [15]:
# input: an image with width:28, height:28, depth:4
# number of filters is defaulted to be 32, each of which is 5*5*4 size. 
a=np.random.randint(100,size=[4,28,28])

# initate two CNN Layers
x = CNNLayer(n=32,filter_shape=(4,5,5),activation="no_act",stride=2,alpha=1)        
x.forward_step(a)

# print out output shape of the the first CNN Layer 
print("input: an image with width:28, height:28, depth:4, stride=2, filter_shape: 5*5*4, number of filters: 32")
print("Shape of the filters of the first CNN Layer:")
print("Order of dimension: ")
print("number of filters, depth of image, width of image, height of image")
print(x.filters.shape)
print("Shape of the output image from the first CNN Layer:")
print(x.output.shape)
y = CNNLayer(n=32,filter_shape=(x.output_shape[0],5,5),activation="relu",stride=2,alpha=1)

# use the first layer's output image as input for the second CNN Layer
y.forward_step(x.output)
print("Shape of the filters of the second CNN Layer:")
print(x.filters.shape)
print("Shape of the output image from the second CNN Layer:")
print(x.output.shape)

# Initiate a FC layer 
n = FCLayer(0.1,10,y.output_shape,np.array([1,1,0,0,0,1,1,1,0,0]))

# predict an image 
n.feed_forward(y.output)
print("Shape of the output from the FC Layer:")
print(n.output.shape)

input: an image with width:28, height:28, depth:4, stride=2, filter_shape: 5*5*4, number of filters: 32
Shape of the filters of the first CNN Layer:
Order of dimension: 
number of filters, depth of image, width of image, height of image
(32, 4, 5, 5)
Shape of the output image from the first CNN Layer:
(32, 13, 13)
Shape of the filters of the second CNN Layer:
(32, 4, 5, 5)
Shape of the output image from the second CNN Layer:
(32, 13, 13)
Shape of the output from the FC Layer:
(10,)


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:73: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# Question 3 

###### I don't want to build another class object based on CNNLayer object. After suffering so much on Question 2, it really kills any desire and willingness to tweak it any furthur. Running out of time, sorry about it. 

In [16]:
# Question 4 - Preparing training images

raw=pd.read_csv("C:\\Users\\zheny\\Downloads\\train.csv")

# create a sample of images that includes images from 0 to 9 
sample_size = 20 # 20 images for each handwritting 
train_dataset = pd.DataFrame([])
for i in range(10):
    train_dataset = pd.concat([train_dataset,raw[raw.iloc[:,0] == i].sample(sample_size)])

labels = train_dataset.iloc[:,0]
y = [] 

for i in labels:
    label_column = [0 for i in range(10)]
    label_column[i] = 1
    y.append(label_column)
    
# y is the label columns, must be numpy array
y = np.array(y)
# X is training images, must be numpy array
X = np.array(train_dataset.iloc[:,1:])

print(X.shape)
print(y.shape)
print("300 images, each of which has 10 labels corresponding to 10 neurons in the FC Layer")

(200, 784)
(200, 10)
300 images, each of which has 10 labels corresponding to 10 neurons in the FC Layer


In [17]:
# Question 4 - training

# Notice: Please be advised that the whole implementation is very sensitive to the 
# weights of the FC Layer, if it's initialized too big/small then the whole back propagation 
# will never converge because its derivative, computed as (f(x+h) - f(x-h))/2h, will always 
# be 0, which in turn causes the delta passed back to previous layer be 0.     

# configuration: 
# number of filters
n = 32 
# filters size: depth 1, heigth&weight 5
filter_size =(1,5,5)
# stride 
stride = 2
# number of node in FC 
node = 10
# learning rate = 1
alpha = 0.0001
# iteration 
t = 10
# first image
image=X[0].reshape([1,28,28])

# first CNNLayer
first_layer = CNNLayer(n = n, filter_shape= filter_size,stride=stride,alpha = alpha)    
first_layer.forward_step(image)  

# second CNNLayer
second_layer = CNNLayer(filter_shape=(first_layer.output_shape[0],5,5),alpha = alpha)
second_layer.forward_step(first_layer.output)

# FC Layer
fc_layer = FCLayer(alpha=alpha,neuron_num=node,weights_shape = second_layer.output_shape,true_label=y[0])

#before = y.filters.sum()
before = fc_layer.neurons[0].weights[0].sum() 

for i in range(t-1):   
    for j in range(len(X)):
        
        print("the", i+1,"Iteration the", j+1,"Image")
        
        image=X[j].reshape([1,28,28])
        fc_layer.set_true_label(y[j])
        
        # forward computation      
        first_layer.forward_step(image)
        second_layer.forward_step(first_layer.output)
        fc_layer.feed_forward(second_layer.output)
        print(fc_layer.output)
        # backward propagation
        fc_layer.back_propagation()
        second_layer.backward_step_from_fc(fc_layer.layer_delta)
        first_layer.backward_step_from_cnn(second_layer.layer_delta)

        # update weights
        first_layer.update_weights_from_cnn()
        second_layer.update_weights_from_fc()
        fc_layer.update_weights()

#print( y.filters.sum()-before)
print(fc_layer.neurons[0].weights[0].sum() -before)
print("Training Finished")

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:73: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


the 1 Iteration the 1 Image
[ 0.73105865  0.73105818  0.73105893  0.73105722  0.73105955  0.73105953
  0.73105934  0.73105887  0.73105837  0.73105856]
the 1 Iteration the 2 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 3 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 4 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 5 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 6 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 7 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 8 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 9 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 10 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 11 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 12 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1 Iteration the 13 Image
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
the 1

KeyboardInterrupt: 

In [61]:
# Question 4 - Creating Testing Dataset of Images 

# create a sample of images that includes images from 0 to 9 
sample_size = 5 #  number of images for each handwritting 

test_dataset = pd.DataFrame([])
for i in range(10):
    test_dataset = pd.concat([test_dataset,raw[raw.iloc[:,0] == i].sample(sample_size)])

# y is the label columns, must be numpy array
test_y = np.array(test_dataset.iloc[:,0])
# X is test images, must be numpy array
test_X = np.array(test_dataset.iloc[:,1:])

print(test_X.shape)
print(test_y.shape)

(50, 784)
(50,)


In [62]:
# Question - Predicting 

result = []

for j in range(len(test_X)):
    
    image=test_X[j].reshape([1,28,28])
    
    # forward computation      
    first_layer.forward_step(image)
    second_layer.forward_step(first_layer.output)
    fc_layer.feed_forward(second_layer.output)
    
    # compiling predictions
    image_prediction = list(fc_layer.output)
    image_prediction.append(test_y[j])
    result.append(image_prediction)

index = ['0','1','2','3','4','5','6','7','8','9']
col_name = ["predict_" + i for i in index]
col_name.append("True Label")

result = pd.DataFrame(result)
result.columns = col_name

# see discussion below
print(result)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:73: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


    predict_0  predict_1  predict_2  predict_3  predict_4  predict_5  \
0         1.0        0.0        0.0        0.0        0.0        0.0   
1         1.0        0.0        0.0        0.0        0.0        0.0   
2         1.0        0.0        0.0        0.0        0.0        0.0   
3         1.0        0.0        0.0        0.0        0.0        0.0   
4         1.0        0.0        0.0        0.0        0.0        0.0   
5         1.0        0.0        0.0        0.0        0.0        0.0   
6         1.0        0.0        0.0        0.0        0.0        0.0   
7         1.0        0.0        0.0        0.0        0.0        0.0   
8         1.0        0.0        0.0        0.0        0.0        0.0   
9         1.0        0.0        0.0        0.0        0.0        0.0   
10        1.0        0.0        0.0        0.0        0.0        0.0   
11        1.0        0.0        0.0        0.0        0.0        0.0   
12        1.0        0.0        0.0        0.0        0.0       

##### discusion: 

Above is the result, first ten columns are if the image is predicted to be 0~9. last column is true labels. As you could see, it does not work totally. Something is wrong with the back propagation. If you take a look at the process of propagation, during the first iteration, input image is 0, the final output quickly changes from 

[ 0.73105865  0.73105818  0.73105893  0.73105722  0.73105955  0.73105953
  0.73105934  0.73105887  0.73105837  0.73105856]

to

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

Then it stays the same for the rest of iteration. I suspect the derivative function does not work well when logistic function gives 1 or 0. The derivative of logistic function does not change at all given a small epsilon. So the derivative function is dead somehow, causing the back propagation process of FC Layer always gives delta as 0. Not sure how to solve it. This is the best I can do.  

# Question 4

My result is doom.. something must be wrong, the accuracy is just like taking a random guess from 10 candidates.

In [3]:
import tensorflow as tf
import cifar10
import sys
import os
import time
import numpy as np

def configuration():
    # Setting up configurations:
    cifar10.maybe_download_and_extract()

    global alpha, image_size,thread_no, lable_numbers, k_size, X, y, weights,biases
    alpha = 0.001
    image_size = 32
    depth = 3
    lable_numbers = 10
    k_size = 2
    X = tf.placeholder(tf.float32, shape=[None, image_size, image_size, depth], name='X')
    y = tf.placeholder(tf.float32, shape=[None, lable_numbers], name='y')

    # initiate weights and biases according to homework specification
    weights = []
    biases= []
    weights.append(tf.Variable(tf.random_normal([3, 3, 3, 64])))
    weights.append(tf.Variable(tf.random_normal([3, 3, 64, 64])))
    weights.append(tf.Variable(tf.random_normal([3, 3, 64, 128])))
    weights.append(tf.Variable(tf.random_normal([3, 3, 128, 128])))
    weights.append(tf.Variable(tf.random_normal([3, 3, 128, 256])))
    weights.append(tf.Variable(tf.random_normal([4*4*256, 256])))

    # for final fc layer
    weights.append(tf.random_normal([256, lable_numbers]))
    
    biases.append(tf.Variable(tf.random_normal([64])))
    biases.append(tf.Variable(tf.random_normal([64])))
    biases.append(tf.Variable(tf.random_normal([128])))
    biases.append(tf.Variable(tf.random_normal([128])))
    biases.append(tf.Variable(tf.random_normal([256])))
    biases.append(tf.Variable(tf.random_normal([256])))

    # for final fc layer
    biases.append(tf.Variable(tf.random_normal([lable_numbers])))

    global final_layer,y_hat,cost,optimizer,initial_global_var,saver,\
           accurate_prediction,accuracy,number_accurate_prediction

    # build a CNN: 
    final_layer, y_hat = build_cnn(X, weights, biases)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=final_layer, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=alpha).minimize(cost)

    # initialization
    initial_global_var = tf.global_variables_initializer()
    saver = tf.train.Saver(tf.global_variables())

    # model evaluator 
    accurate_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(accurate_prediction, tf.float32))
    number_accurate_prediction = tf.reduce_sum(tf.cast(accurate_prediction, tf.float32))


# make some aux function to call functions within tensorflow class 
def pool_layer(x, k_size=2):
    
    # print(tf.nn.max_pool(x,ksize=[1,k_size,k_size,1],strides=[1,k_size,k_size,1]))
    return tf.nn.max_pool(x,ksize=[1,k_size,k_size,1],strides=[1,k_size,k_size,1],padding='SAME')

def conv_layer(x, weights, bias, strides=1):
    # processing input array
    x = tf.nn.conv2d(x, weights, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, bias)
    # print(x)
    return tf.nn.relu(x)

def fc_layer(input,weight,bias,size=[-1, 4*4*256]):
    # fully connected layer
    layer = tf.reshape(input, size)
    layer = tf.add(tf.matmul(layer, weight), bias)
    return tf.nn.relu(layer)

# define a CNN model
# notice: this is like a hard code function that only works for
#         the specification of the 10 layers CNN specified by
#         the homework requirement.
# returns final predict and layer for debug
                   
def build_cnn(x, weights, biases):

    # according to the specification of the question

    conv1 = conv_layer(X, weights[0], biases[0])
    conv2 = conv_layer(conv1, weights[1], biases[1])
    
    pool1 = pool_layer(conv2)
    conv3 = conv_layer(pool1, weights[2], biases[2])
    conv4 = conv_layer(conv3, weights[3], biases[3])
    pool2 = pool_layer(conv4)
    conv5 = conv_layer(pool2, weights[4], biases[4])
    pool3 = pool_layer(conv5)
    
    fc1 = fc_layer(pool3,weights[5], biases[5])

    fc2 = tf.add(tf.matmul(fc1,weights[6]),biases[6])
    result = tf.nn.softmax(fc2)

    return fc2,result


# number of batch to train on 
def train(batch = 5,iteration = 100):

    number_of_batch = 100
    batch_per_iter = int(50000/number_of_batch)
    train_images, train_cls, train_labels = cifar10.load_training_data()
    
    with tf.Session() as thread:
        thread.run(initial_global_var)
        
        for i in range(batch):
            print("Now: "+str(i)+" iteration")

            step_size = i%batch_per_iter 

            current_x, current_y = train_images[step_size*number_of_batch:(step_size + 1)\
                        *number_of_batch],train_labels[step_size*number_of_batch:(step_size+1)*number_of_batch]
            
            thread.run(optimizer, feed_dict={X: current_x, y: current_y})

            total_loss = 0.0
            total_accuracy = 0.0
            
            for j in range(iteration):
                current_loss, current_accuracy = thread.run([cost,accuracy],\
                        feed_dict={X:train_images[j*number_of_batch:(j+1)*number_of_batch],\
                        y:train_labels[j*number_of_batch:(j+1)*number_of_batch]})

                total_loss=total_loss+current_loss
                total_accuracy = total_accuracy+current_accuracy

            total_loss = total_loss/iteration
            total_accuracy = total_accuracy/iteration

            print("Batch "+ str(i)  + ", current raining Accuracy= ", total_accuracy)
        saver.save(thread, os.path.join(os.getcwd()))
          
    print("training done\n")


# to speed up, the number of images to be tested is down to 10
def test(test_image_num = 10):

    test_images, test_cls, test_labels = cifar10.load_test_data()
    number_of_batch = 100
    
    with tf.Session() as thread:

        model = tf.train.import_meta_graph(os.path.join(os.getcwd(),"model.meta"))
        model.restore(thread, os.path.join(os.getcwd(),"model"))

        accuracy = []
        
        for i in range(test_image_num):
        
            accuracy.append(thread.run(number_accurate_prediction,feed_dict={X:test_images[i*number_of_batch:(i+1)*number_of_batch],\
                            y:test_labels[i*number_of_batch:(i+1)*number_of_batch]}))

        print("prediction accuracy of the first testing images: ", np.array(accuracy)/100.0)
        

def main(args):

    if args[1] =="train":
        print("To illastrate the whole program, iteration and batch size has been reduce lower enough to speed up the process time.")
        configuration()                    
        train()
        
    elif args[1] == "test":
        test()

    else:
        print("This program takes 1 argument: hw5-tf.py -test/-train")
        return
                            
    return
                            
main(['hw.py','train'])
main(['hw.py','test'])
#main(sys.argv)

ImportError: No module named 'tensorflow'